### 03. Transform Validated Data to Hourly Time Series Format

This notebook loads validated Citi Bike daily station-level data and converts it into an **hourly-level time series format** suitable for modeling.

The output is a `.csv` file per location with columns:
- `pickup_datetime`
- `location_id`
- `rides`

These are saved to:  
`data/features/location_<ID>.csv`


In [1]:
import os
import sys

# Set the working directory to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(project_root)

# Confirm it worked
print("✅ Current working directory set to:", os.getcwd())


✅ Current working directory set to: /Users/snehitha/citi_bike_project


In [2]:
import pandas as pd

validated_dir = "data/validated"
files = sorted([f for f in os.listdir(validated_dir) if f.endswith(".csv")])

print(f"✅ Found {len(files)} validated files")
files[:3]  # Show first 3


✅ Found 12 validated files


['JC-202401-citibike-tripdata.csv',
 'JC-202402-citibike-tripdata.csv',
 'JC-202403-citibike-tripdata.csv']

In [3]:
# Combine all validated CSVs into a single DataFrame
dfs = []

for fname in files:
    fpath = os.path.join(validated_dir, fname)
    df = pd.read_csv(fpath, parse_dates=["started_at", "ended_at"])
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
print(f"✅ Combined DataFrame shape: {df_all.shape}")
df_all.head()


✅ Combined DataFrame shape: (1039139, 14)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member,0 days 00:14:37
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member,0 days 00:03:28
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,0 days 00:03:44
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,0 days 00:03:22
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member,0 days 00:04:21


In [4]:
# Add pickup_hour by flooring to the nearest hour
df_all["pickup_hour"] = df_all["started_at"].dt.floor("H")

# Group by pickup_hour and start_station_id and count rides
ride_counts = (
    df_all.groupby(["pickup_hour", "start_station_id"])
    .size()
    .reset_index(name="ride_count")
)

ride_counts.head()


,pickup_hour,start_station_id,ride_count
0,2024-01-01,HB102,5
1,2024-01-01,HB103,1
2,2024-01-01,HB105,3
3,2024-01-01,HB201,1
4,2024-01-01,HB202,1


In [5]:
ride_counts["pickup_hour"].dt.strftime("%Y-%m-%d %H:%M:%S").unique()[:5]


array(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
       '2024-01-01 02:00:00', '2024-01-01 03:00:00',
       '2024-01-01 04:00:00'], dtype=object)

In [6]:
# Pivot into time series format: each row = pickup_hour, each column = start_station_id
ts_df = (
    ride_counts.pivot(index="pickup_hour", columns="start_station_id", values="ride_count")
    .fillna(0)
    .astype(int)
)

print(f"✅ Time series shape: {ts_df.shape}")
ts_df.head()


✅ Time series shape: (8748, 205)


start_station_id,4074.14,4298.05,4461.07,4762.05,4977.03,4993.02,4993.15,5024.10,5033.01,5105.01,...,JC102,JC103,JC104,JC105,JC107,JC108,JC109,JC110,JC115,JC116
pickup_hour,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,2,0,3,11
2024-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,4,0,2,5,0,0,4,1,4,2
2024-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,2,2,4,1,0,0,2,1,6,4
2024-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,2,2,1,0,0,1,2,2,0
2024-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0


In [7]:
# Save the time series DataFrame
ts_df.to_csv("data/ts_data.csv")
print("✅ Time series data saved to 'data/ts_data.csv'")


✅ Time series data saved to 'data/ts_data.csv'
